# Model Comparison

In this example you will learn how to compare different models with TruLens.

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/truera/trulens/blob/main/trulens_eval/examples/expositional/models/multimodal_eval.ipynb)

## Setup
### Add API keys
For this quickstart you will need Open AI and Huggingface keys

In [ ]:
from IPython.display import Image
import io
import requests

import os
os.environ["REPLICATE_API_TOKEN"] = "..."

In [ ]:
import replicate

def image_generator(prompt):
    output = replicate.run(
  "stability-ai/stable-diffusion:27b93a2413e7f36cd83da926f3656280b2931564ff050bf9575f1fdf9bcd7478",
  input={"prompt": prompt}
)
    return output[0]

In [ ]:
image = image_generator("An iguana on the beach, pointilism")

In [ ]:
Image(image)

In [ ]:
from trulens_eval import Replicate
from pydantic import BaseModel, Field

import json

In [ ]:
class Multimodal(Replicate):
    model_engine: str = "yorickvp/llava-13b:2facb4a474a0462c15041b78b1ad70952ea46b5ec6ad29583c0b29dbd4249591"

    def __init__(self, model_engine: str = model_engine):
        super().__init__()
        self.model_engine = model_engine

    def _handle_streaming(self, output) -> str:
        full_output = []
        for item in output:
            print(item, end="")
            full_output.append(item)
        # Join the list of strings into a single string
        output_str = ''.join(full_output)
        # Replace single quotes with double quotes
        output_str = output_str.replace("'", '"')
        return output_str  # Add this line

    def _eval_image(self, image_url: str, eval_prompt: str) -> dict:
        response = requests.get(image_url)
        file_bytes = response.content

        file_like_object = io.BytesIO(file_bytes)

        output = replicate.run(
            self.model_engine,
            input={"image": file_like_object, "prompt" : eval_prompt}
        )
        output_str = self._handle_streaming(output)
        
        return json.loads(output_str)

    def iguana_score(self, image_url: str) -> float:
        eval_prompt = eval_prompt = "Is the image of an iguana? Rate from 0 to 10 and respond only with the rating as a float."
        score = self._eval_image(image_url = image_url, eval_prompt = eval_prompt)
        return float(score)

    def pointilism_score(self, image_url: str) -> float:
        eval_prompt = "Is the image in pointilism style? Rate from 0 to 10 and respond only with the rating as a float."
        score = self._eval_image(image_url = image_url, eval_prompt = eval_prompt)
        return float(score)

In [ ]:
Tru().reset_database()

In [ ]:
lmm = Multimodal()

In [ ]:
from trulens_eval import Feedback
f_iguana_score = Feedback(lmm.iguana_score).on_output()
f_pointilism_score = Feedback(lmm.pointilism_score).on_output()

feedbacks = [f_iguana_score, f_pointilism_score]

In [ ]:
from trulens_eval import TruBasicApp

tru_multimodal_recorder = TruBasicApp(image_generator, app_id="Image Generator", feedbacks=feedbacks)

In [ ]:
with tru_multimodal_recorder as recording:
    tru_multimodal_recorder.app("An iguana on the beach, pointilism")

In [ ]:
from trulens_eval import Tru

Tru().run_dashboard()